# Exploring Simulated Data Products

The purpose of this notebook is to explore the simulated event lightcurves and parameter sets generated by Anibal Verala. 

In [6]:
from os import path, getcwd
from sys import path as pythonpath
pythonpath.append(path.join(getcwd(), '..'))
import numpy as np
import utils
from pyLIMA import event
from pyLIMA import telescopes
from pyLIMA.fits import TRF_fit
from pyLIMA.models import PSPL_model

The test_data directory contains examples of Anibal's data, so lets load one of those files using the read_data routine from his package.

In [2]:
event_file = path.join(getcwd(), '..', 'test_data', 'Event_18.h5')

info_dataset, pyLIMA_parameters, phot_data = utils.read_data(event_file)

print(info_dataset)
print(pyLIMA_parameters)
print(phot_data)

['18', '/home/anibal-pc/roman_rubin/TRILEGAL/PB_planet_split_1.csv', ['third_caustic', [0, 0]]]
{'alpha': np.float64(2.170221123244217), 'fblend_W149': np.float64(206.28179213395828), 'fblend_g': np.float64(100.45169468197552), 'fblend_i': np.float64(223.6983332752606), 'fblend_r': np.float64(69.10845649953637), 'fblend_u': np.float64(2.493525365117888), 'fblend_y': np.float64(87.62721758298721), 'fblend_z': np.float64(173.00884416162543), 'fsource_W149': np.float64(317.17398829050336), 'fsource_g': np.float64(114.33135273107756), 'fsource_i': np.float64(262.4849251897241), 'fsource_r': np.float64(380.0504336063954), 'fsource_u': np.float64(4.933245072029084), 'fsource_y': np.float64(131.55231680142592), 'fsource_z': np.float64(230.63654880597986), 'ftotal_W149': np.float64(523.4557804244616), 'ftotal_g': np.float64(214.78304741305308), 'ftotal_i': np.float64(486.1832584649847), 'ftotal_r': np.float64(449.1588901059318), 'ftotal_u': np.float64(7.426770437146972), 'ftotal_y': np.float64

From this we can create a list of pyLIMA Telescopes in order to fit the model. 

In [48]:
lsst_filters = ['u', 'g', 'r', 'i', 'z', 'y']
roman_filters = ['W149']

tel_list = []
# Note the use of Gaia as the name of the spacecraft rather than Roman. 
# This is because the name is used by PyLIMA to search JPL Horizons for the orbital ephemeris information
# when computing the parallax.  Roman isn't yet in JPL Horizons, since it hasn't launched yet. 
for f in roman_filters:
    lightcurve = np.column_stack([ phot_data[f][col].data for col in ['time', 'mag', 'err_mag'] ])

    if len(lightcurve) > 0:
        tel = telescopes.Telescope(name='Roman_' + f, 
                                     camera_filter=f, 
                                     lightcurve = lightcurve,
                                     lightcurve_names = ['time','mag','err_mag'],
                                     lightcurve_units = ['JD','mag','err_mag'],
                                     location = 'Space')
    tel.spacecraft_name = 'James Webb Space Telescope'
    tel_list.append(tel)

for f in lsst_filters:
    lightcurve = np.column_stack([ phot_data[f][col].data for col in ['time', 'mag', 'err_mag'] ])
    
    if len(lightcurve) > 0:
        tel = telescopes.Telescope(name='LSST_' + f, 
                                     camera_filter=f, 
                                     lightcurve = lightcurve,
                                     lightcurve_names = ['time','mag','err_mag'],
                                     lightcurve_units = ['JD','mag','err_mag'],
                                     location = 'Earth')
        tel_list.append(tel)
    
tel_list

In order for PyLIMA to fit the parallax, the event needs to be created with parallax parameters that include the timestamp.  Here we use the timestamp of the maximum flux value for each telescope.  

In [49]:
max_flux = []
for tel in tel_list:
    max_flux.append(tel.lightcurve['time'][np.array(tel.lightcurve['mag']).argmin()].value)
max_flux

[np.float64(2462552.1458332045),
 np.float64(2462626.7279896727),
 np.float64(2462626.703556429),
 np.float64(2462571.8822298953),
 np.float64(2462553.891815335),
 np.float64(2462698.724820037)]

Let's try a PSPL model fit to start with.

In [50]:
# One key item missing from the stored event information seems to be the coordinates of the event.  For the purpose of this example, 
# I've used the coordinates of a field near the central Bulge.
event18 = event.Event(ra=270.766667, dec=-28.61083333)
event18.name = 'RR_event_18'
event18.telescopes = tel_list

In [51]:
pspl = PSPL_model.PSPLmodel(event18, parallax=['Full', np.mean(max_flux)])
trf  = TRF_fit.TRFfit(pspl)

/data/software/pylima_venv/lib/python3.12/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "utctai" yielded 20736 of "dubious year (Note 3)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)
/data/software/pylima_venv/lib/python3.12/site-packages/erfa/core.py:133: ErfaWarning: ERFA function "taiutc" yielded 20736 of "dubious year (Note 4)"
  warn(f'ERFA function "{func_name}" yielded {wmsg}', ErfaWarning)


ValueError: Unknown target (V37). Maybe try different id_type?